In [324]:
import pandas as pd
import numpy as np
import math
import datetime

In [325]:
train = pd.read_csv('data/train_users_2.csv')
test = pd.read_csv('data/test_users.csv')
countries = pd.read_csv('data/countries.csv')
user_demo = pd.read_csv('data/age_gender_bkts.csv')
sessions = pd.read_csv('data/sessions.csv')
#changing into datetime
train.date_account_created = pd.to_datetime(train.date_account_created)
train.timestamp_first_active = pd.to_datetime(train.timestamp_first_active, format = "%Y%m%d%H%M%S")
train.date_first_booking = pd.to_datetime(train.date_first_booking)
test.timestamp_first_active = pd.to_datetime(test.timestamp_first_active, format = "%Y%m%d%H%M%S")
test.date_account_created = pd.to_datetime(test.date_account_created)

train_destination = train.iloc[:,-1]

In [326]:
#sessions grouping by user

#Group by user_id, aggregate by number of counts (counting device_type as it is never NA), 
#and total sum of elapsed time in seconds
group_sessions = sessions.groupby("user_id").agg({'device_type':'count', 'secs_elapsed':'sum'})
#rename columns
group_sessions.columns = ['sum_secs_elapsed', 'counts']
#group by variable turns into index, I'm reseting the index and putting user_id back as a column
group_sessions.reset_index(level=0, inplace=True)

In [7]:
group_sessions.head() 

,user_id,sum_secs_elapsed,counts
0,00023iyk9l,867896.0,40
1,0010k6l0om,586543.0,63
2,001wyh0pz8,282965.0,90
3,0028jgx1x1,297010.0,31
4,002qnbzfs5,6487080.0,789


In [327]:
#bucket all ages into format that user_demo is in for age
def agebuckets(ages):
    ageless =  [i for i in range(5,101,5)] # 5, 10, 15, 20...95, 100
    buckets = ['%d-%d' %(i, i+4) for i in range(0,100,5)] # 0-4, 5-9, 10-14...90-94, 95-99
    newlist = []
    for i in range(len(ages)):
        if math.isnan(ages[i]):
            newlist.append('NA')
        elif ages[i] <ageless[0]:
            newlist.append(buckets[0])
        elif ages[i] < ageless[1]:
            newlist.append(buckets[1])
        elif ages[i] < ageless[2]:
            newlist.append(buckets[2])
        elif ages[i] < ageless[3]:
            newlist.append(buckets[3])
        elif ages[i] < ageless[4]:
            newlist.append(buckets[4])
        elif ages[i] < ageless[5]:
            newlist.append(buckets[5])
        elif ages[i] < ageless[6]:
            newlist.append(buckets[6])
        elif ages[i] < ageless[7]:
            newlist.append(buckets[7])
        elif ages[i] < ageless[8]:
            newlist.append(buckets[8])
        elif ages[i] < ageless[9]:
            newlist.append(buckets[9])
        elif ages[i] < ageless[10]:
            newlist.append(buckets[10])
        elif ages[i] < ageless[11]:
            newlist.append(buckets[11])
        elif ages[i] < ageless[12]:
            newlist.append(buckets[12]) 
        elif ages[i] < ageless[13]:
            newlist.append(buckets[13]) 
        elif ages[i] < ageless[14]:
            newlist.append(buckets[14])
        elif ages[i] < ageless[15]:
            newlist.append(buckets[15])
        elif ages[i] < ageless[16]:
            newlist.append(buckets[16])
        elif ages[i] < ageless[17]:
            newlist.append(buckets[17])
        elif ages[i] < ageless[18]:
            newlist.append(buckets[18])
        elif ages[i] < ageless[19]:
            newlist.append(buckets[19]) 
        else:
            newlist.append('100+')
    return newlist

In [9]:
train.age[0:20] #to be deleted

0      NaN
1     38.0
2     56.0
3     42.0
4     41.0
5      NaN
6     46.0
7     47.0
8     50.0
9     46.0
10    36.0
11    47.0
12     NaN
13    37.0
14    36.0
15    33.0
16     NaN
17    31.0
18     NaN
19    29.0
Name: age, dtype: float64

In [328]:
train.age = agebuckets(train.age)
test.age = agebuckets(test.age)

In [11]:
train.age[0:20] 

0        NA
1     35-39
2     55-59
3     40-44
4     40-44
5        NA
6     45-49
7     45-49
8     50-54
9     45-49
10    35-39
11    45-49
12       NA
13    35-39
14    35-39
15    30-34
16       NA
17    30-34
18       NA
19    25-29
Name: age, dtype: object

In [329]:
def timedif(L1, L2):
    timediflist = []
    for i in range(len(L1)):
        try:
            if (L1[i]-L2[i]).days <= -1:#datetime.timedelta(days=0):
                timediflist.append('before')
            elif (L1[i]-L2[i]).days ==0: #datetime.timedelta(days=1):
                timediflist.append('same day')
            else:
                timediflist.append('greater 1 day')
        except:
            timediflist.append('NA')
            
    return timediflist

In [13]:
np.unique(timedif(train.date_first_booking, train.date_account_created)) #testing to be deleted

array(['NA', 'before', 'greater 1 day', 'same day'], 
      dtype='|S13')

In [14]:
np.unique(timedif(test.date_first_booking, test.date_account_created)) #testing to be deleted 

array(['NA'], 
      dtype='|S2')

In [15]:
np.unique(timedif(train.date_first_booking, train.timestamp_first_active)) # testing to be deleted 

array(['NA', 'before', 'greater 1 day', 'same day'], 
      dtype='|S13')

In [16]:
np.unique(timedif(test.date_first_booking, test.timestamp_first_active)) # testing to be deleted 

array(['NA'], 
      dtype='|S2')

In [330]:
#adding time lag columns
train['lag_account_created'] = timedif(train.date_first_booking, train.date_account_created)
train['lag_first_active'] = timedif(train.date_first_booking, train.timestamp_first_active)
train['lag_account_created_first_active'] = timedif(train.date_account_created, train.timestamp_first_active)
test['lag_account_created_first_active'] = timedif(test.date_account_created, test.timestamp_first_active)

In [331]:
#given the train data gender, age, and country_desination produce the corresponding population in thousands
population_in_thous = []
for i in range(train.shape[0]):
    if train.gender[i] == '-unknown-' or train.gender[i] == 'OTHER' or train.age[i] == 'NA' or train.country_destination[i] == 'NDF' or train.country_destination[i] == 'other': 
        population_in_thous.append('NA')
    else:
        genders = user_demo.loc[user_demo.gender == train.gender[i].lower(),:] 

        dests = genders.loc[genders.country_destination == train.country_destination[i] ,:]    
        population_in_thous.append(float((dests.loc[dests.age_bucket == train.age[i], 'population_in_thousands'])))
        
population_in_thous[0:10]

['NA', 'NA', 11264.0, 'NA', 'NA', 'NA', 10659.0, 10659.0, 11413.0, 'NA']

In [20]:
genders.head()

,age_bucket,country_destination,gender,population_in_thousands,year
21,100+,AU,female,4.0,2015.0
22,95-99,AU,female,25.0,2015.0
23,90-94,AU,female,94.0,2015.0
24,85-89,AU,female,179.0,2015.0
25,80-84,AU,female,252.0,2015.0


In [332]:
#merging gender age bucket with train data
train['population_in_thousands'] = population_in_thous

In [333]:
#merging with grouped sessions and countries, **note most of training data is not in sessions. see below 
test = pd.merge(test, group_sessions, left_on='id', right_on ='user_id', how='left')
train = pd.merge(train, group_sessions, left_on='id', right_on ='user_id', how='left')
test = test.drop('user_id', 1)
train = train.drop('user_id', 1)
train_m = pd.merge(train, countries, on='country_destination', how='left')
print train_m.iloc[0:5, 0:10] #to be deleted?
print train_m.iloc[0:5, 10:]  # to be deleted?

           id date_account_created timestamp_first_active date_first_booking  \
0  gxn3p5htnn           2010-06-28    2009-03-19 04:32:55                NaT   
1  820tgsjxq7           2011-05-25    2009-05-23 17:48:09                NaT   
2  4ft3gnwmtx           2010-09-28    2009-06-09 23:12:47         2010-08-02   
3  bjjt8pjhuk           2011-12-05    2009-10-31 06:01:29         2012-09-08   
4  87mebub9p4           2010-09-14    2009-12-08 06:11:05         2010-02-18   

      gender    age signup_method  signup_flow language affiliate_channel  
0  -unknown-     NA      facebook            0       en            direct  
1       MALE  35-39      facebook            0       en               seo  
2     FEMALE  55-59         basic            3       en            direct  
3     FEMALE  40-44      facebook            0       en            direct  
4  -unknown-  40-44         basic            0       en            direct  
  affiliate_provider first_affiliate_tracked signup_app first_d

In [2]:
import pandas as pd
train_m = pd.read_csv('train_starting.csv')

In [3]:
#remove extra space from column name
train_m = train_m.rename(columns={'destination_language ': 'destination_language'})

In [4]:
train_m.head()

,Unnamed: 0,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,...,bookings,population_in_thousands,sum_secs_elapsed,counts,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,0,gxn3p5htnn,2010-06-28 00:00:00,2009-03-19 04:32:55,NaN,NaN,NaN,facebook,0,en,...,NB,NB,NaN,NaN,NB,NB,NB,NB,NB,NB
1,1,820tgsjxq7,2011-05-25 00:00:00,2009-05-23 17:48:09,NaN,MALE,35-39,facebook,0,en,...,NB,NB,NaN,NaN,NB,NB,NB,NB,NB,NB
2,2,4ft3gnwmtx,2010-09-28 00:00:00,2009-06-09 23:12:47,2010-08-02 00:00:00,FEMALE,55-59,basic,3,en,...,waited,11264.0,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,eng,0.0
3,3,bjjt8pjhuk,2011-12-05 00:00:00,2009-10-31 06:01:29,2012-09-08 00:00:00,FEMALE,40-44,facebook,0,en,...,waited,2458.8000000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,87mebub9p4,2010-09-14 00:00:00,2009-12-08 06:11:05,2010-02-18 00:00:00,NaN,40-44,basic,0,en,...,waited,NaN,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,eng,0.0


In [5]:
train_m.columns

Index([u'Unnamed: 0', u'id', u'date_account_created',
       u'timestamp_first_active', u'date_first_booking', u'gender', u'age',
       u'signup_method', u'signup_flow', u'language', u'affiliate_channel',
       u'affiliate_provider', u'first_affiliate_tracked', u'signup_app',
       u'first_device_type', u'first_browser', u'country_destination',
       u'lag_account_created', u'lag_first_active',
       u'lag_account_created_first_active', u'bookings',
       u'population_in_thousands', u'sum_secs_elapsed', u'counts',
       u'lat_destination', u'lng_destination', u'distance_km',
       u'destination_km2', u'destination_language',
       u'language_levenshtein_distance'],
      dtype='object')

In [6]:
train_m.describe()

/usr/local/lib/python2.7/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Unnamed: 0,signup_flow,sum_secs_elapsed,counts
count,213451.000000,213451.000000,7.263100e+04,73815.000000
mean,106725.000000,3.267387,1.538919e+06,75.024819
std,61618.140492,7.637707,1.918849e+06,112.543174
min,0.000000,0.000000,0.000000e+00,1.000000
25%,53362.500000,0.000000,NaN,NaN
50%,106725.000000,0.000000,NaN,NaN
75%,160087.500000,0.000000,NaN,NaN
max,213450.000000,25.000000,3.822136e+07,2644.000000


In [7]:
#remove id, time format columns and categorical columns
excl = list(train_m.columns)
toremove = ['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 
            'population_in_thousands', 'gender', 'age', 'country_destination',
            'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 
            'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 
            'lag_account_created', 'lag_first_active', 'lag_account_created_first_active', 'destination_language'] 
map(lambda x: excl.remove(x), toremove)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
train_m.loc[:, excl]

,Unnamed: 0,signup_flow,bookings,sum_secs_elapsed,counts,lat_destination,lng_destination,distance_km,destination_km2,language_levenshtein_distance
0,0,0,NB,NaN,NaN,NB,NB,NB,NB,NB
1,1,0,NB,NaN,NaN,NB,NB,NB,NB,NB
2,2,3,waited,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
3,3,0,waited,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,waited,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
5,5,0,early,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
6,6,0,waited,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
7,7,0,waited,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
8,8,0,waited,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0
9,9,0,early,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0


In [9]:
train_x = train_m.loc[:, excl]

In [10]:
print pd.get_dummies(train_m.gender).head()
print pd.get_dummies(train_m.age).head()
print pd.get_dummies(train_m.signup_method).head()
print pd.get_dummies(train_m.language).head()
print pd.get_dummies(train_m.affiliate_channel).head()
print pd.get_dummies(train_m.affiliate_provider).head()
print pd.get_dummies(train_m.first_affiliate_tracked).head()
print pd.get_dummies(train_m.signup_app).head()
print pd.get_dummies(train_m.first_device_type).head()
print pd.get_dummies(train_m.first_browser).head()
print pd.get_dummies(train_m.country_destination).head()
print pd.get_dummies(train_m.lag_account_created).head()
print pd.get_dummies(train_m.lag_first_active).head()
print pd.get_dummies(train_m.lag_account_created_first_active).head()
print pd.get_dummies(train_m.destination_language).head()
#leave date_account_created
#leave timestamp_first_active
#leave date_first_booking
#leave population_in_thousands

   FEMALE  MALE  OTHER
0     0.0   0.0    0.0
1     0.0   1.0    0.0
2     1.0   0.0    0.0
3     1.0   0.0    0.0
4     0.0   0.0    0.0
   0-4  100+  15-19  20-24  25-29  30-34  35-39  40-44  45-49  5-9  50-54  \
0  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0   
1  0.0   0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0  0.0    0.0   
2  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0   
3  0.0   0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  0.0    0.0   
4  0.0   0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0  0.0    0.0   

   55-59  60-64  65-69  70-74  75-79  80-84  85-89  90-94  95-99  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
2    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0

In [11]:
print pd.get_dummies(train_m.gender).columns
print pd.get_dummies(train_m.age).columns
print pd.get_dummies(train_m.signup_method).columns
print pd.get_dummies(train_m.language).columns
print pd.get_dummies(train_m.affiliate_channel).columns
print pd.get_dummies(train_m.affiliate_provider).columns
print pd.get_dummies(train_m.first_affiliate_tracked).columns
print pd.get_dummies(train_m.signup_app).columns
print pd.get_dummies(train_m.first_device_type).columns
print pd.get_dummies(train_m.first_browser).columns
print pd.get_dummies(train_m.country_destination).columns
print pd.get_dummies(train_m.lag_account_created).columns
print pd.get_dummies(train_m.lag_first_active).columns
print pd.get_dummies(train_m.lag_account_created_first_active).columns
print pd.get_dummies(train_m.destination_language).columns

Index([u'FEMALE', u'MALE', u'OTHER'], dtype='object')
Index([u'0-4', u'100+', u'15-19', u'20-24', u'25-29', u'30-34', u'35-39',
       u'40-44', u'45-49', u'5-9', u'50-54', u'55-59', u'60-64', u'65-69',
       u'70-74', u'75-79', u'80-84', u'85-89', u'90-94', u'95-99'],
      dtype='object')
Index([u'basic', u'facebook', u'google'], dtype='object')
Index([u'ca', u'cs', u'da', u'de', u'el', u'en', u'es', u'fi', u'fr', u'hr',
       u'hu', u'id', u'is', u'it', u'ja', u'ko', u'nl', u'no', u'pl', u'pt',
       u'ru', u'sv', u'th', u'tr', u'zh'],
      dtype='object')
Index([u'api', u'content', u'direct', u'other', u'remarketing', u'sem-brand',
       u'sem-non-brand', u'seo'],
      dtype='object')
Index([u'baidu', u'bing', u'craigslist', u'daum', u'direct',
       u'email-marketing', u'facebook', u'facebook-open-graph', u'google',
       u'gsp', u'meetup', u'naver', u'other', u'padmapper', u'vast', u'wayn',
       u'yahoo', u'yandex'],
      dtype='object')
Index([u'linked', u'local ops',

In [13]:
#gender
train_x[list(pd.get_dummies(train_m.gender).columns)[0]] = pd.get_dummies(train_m.gender).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.gender).columns))):
    train_x[list(pd.get_dummies(train_m.gender).columns)[i]] = pd.get_dummies(train_m.gender).iloc[:,i]
train_x['gender_unknown'] = pd.get_dummies(train_m.gender).iloc[:,0]
train_x['gender_other'] = pd.get_dummies(train_m.gender).iloc[:,3]

#age
train_x[list(pd.get_dummies(train_m.age).columns)[0]] = pd.get_dummies(train_m.age).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.age).columns))):
    train_x[list(pd.get_dummies(train_m.age).columns)[i]] = pd.get_dummies(train_m.age).iloc[:,i]
train_x['age_NA'] = pd.get_dummies(train_m.age).iloc[:,20]

#signup_method
train_x[list(pd.get_dummies(train_m.signup_method).columns)[0]] = pd.get_dummies(train_m.signup_method).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.signup_method).columns))):
    train_x[list(pd.get_dummies(train_m.signup_method).columns)[i]] = pd.get_dummies(train_m.signup_method).iloc[:,i]

#language
train_x[list(pd.get_dummies(train_m.language).columns)[0]] = pd.get_dummies(train_m.language).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.language).columns))):
    train_x[list(pd.get_dummies(train_m.language).columns)[i]] = pd.get_dummies(train_m.language).iloc[:,i]

#affiliate_channel
train_x[list(pd.get_dummies(train_m.affiliate_channel).columns)[0]] = pd.get_dummies(train_m.affiliate_channel).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.affiliate_channel).columns))):
    train_x[list(pd.get_dummies(train_m.affiliate_channel).columns)[i]] = pd.get_dummies(train_m.affiliate_channel).iloc[:,i]

#affiliate_provider
train_x[list(pd.get_dummies(train_m.affiliate_provider).columns)[0]] = pd.get_dummies(train_m.affiliate_provider).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.affiliate_provider).columns))):
    train_x[list(pd.get_dummies(train_m.affiliate_provider).columns)[i]] = pd.get_dummies(train_m.affiliate_provider).iloc[:,i]

#first_affiliate_tracked
train_x[list(pd.get_dummies(train_m.first_affiliate_tracked).columns)[0]] = pd.get_dummies(train_m.first_affiliate_tracked).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_affiliate_tracked).columns))):
    train_x[list(pd.get_dummies(train_m.first_affiliate_tracked).columns)[i]] = pd.get_dummies(train_m.first_affiliate_tracked).iloc[:,i]

#signup_app
train_x[list(pd.get_dummies(train_m.signup_app).columns)[0]] = pd.get_dummies(train_m.signup_app).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.signup_app).columns))):
    train_x[list(pd.get_dummies(train_m.signup_app).columns)[i]] = pd.get_dummies(train_m.signup_app).iloc[:,i]

#first_device_type
train_x[list(pd.get_dummies(train_m.first_device_type).columns)[0]] = pd.get_dummies(train_m.first_device_type).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_device_type).columns))):
    train_x[list(pd.get_dummies(train_m.first_device_type).columns)[i]] = pd.get_dummies(train_m.first_device_type).iloc[:,i]

#first_browser
train_x[list(pd.get_dummies(train_m.first_browser).columns)[0]] = pd.get_dummies(train_m.first_browser).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.first_browser).columns))):
    train_x[list(pd.get_dummies(train_m.first_browser).columns)[i]] = pd.get_dummies(train_m.first_browser).iloc[:,i]
train_x['first_browser_unknown'] = pd.get_dummies(train_m.first_browser).iloc[:,0]
    
#country_destination
train_x[list(pd.get_dummies(train_m.country_destination).columns)[0]] = pd.get_dummies(train_m.country_destination).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.country_destination).columns))):
    train_x[list(pd.get_dummies(train_m.country_destination).columns)[i]] = pd.get_dummies(train_m.country_destination).iloc[:,i]
train_x['country_dest_other'] = pd.get_dummies(train_m.country_destination).iloc[:,11]

#lag_account_created
train_x['lag_account_created_NA'] = pd.get_dummies(train_m.lag_account_created).iloc[:,0]
train_x['lag_account_created_before'] = pd.get_dummies(train_m.lag_account_created).iloc[:,1]
train_x['lag_account_created_greater1'] = pd.get_dummies(train_m.lag_account_created).iloc[:,2]
train_x['lag_account_created_sameday'] = pd.get_dummies(train_m.lag_account_created).iloc[:,2]

#lag_first_active
train_x['lag_first_active_NA'] = pd.get_dummies(train_m.lag_first_active).iloc[:,0]
train_x['lag_first_active_before'] = pd.get_dummies(train_m.lag_first_active).iloc[:,1]
train_x['lag_first_active_greater1'] = pd.get_dummies(train_m.lag_first_active).iloc[:,2]
train_x['lag_first_active_sameday'] = pd.get_dummies(train_m.lag_first_active).iloc[:,2]

#lag_account_created_first_active
train_x['lag_account_created_first_active_before'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,0]
train_x['lag_account_created_first_active_greater1'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,1]
train_x['lag_account_created_first_active_sameday'] = pd.get_dummies(train_m.lag_account_created_first_active).iloc[:,2]

#destination_language
train_x[list(pd.get_dummies(train_m.destination_language).columns)[0]] = pd.get_dummies(train_m.destination_language).iloc[:,0]
for i in range(len(list(pd.get_dummies(train_m.destination_language).columns))):
    train_x[list(pd.get_dummies(train_m.destination_language).columns)[i]] = pd.get_dummies(train_m.destination_language).iloc[:,i]

train_x = train_x.drop('-unknown-', 1)
train_x = train_x.drop('OTHER', 1)
train_x = train_x.drop('NA', 1)

IndexError: single positional indexer is out-of-bounds

In [353]:
pd.get_dummies(train_m.destination_language).columns

Index([u'deu', u'eng', u'fra', u'ita', u'nld', u'por', u'spa'], dtype='object')

In [376]:
list(train_x.columns)

['signup_flow',
 'sum_secs_elapsed',
 'counts',
 'lat_destination',
 'lng_destination',
 'distance_km',
 'destination_km2',
 'language_levenshtein_distance',
 'FEMALE',
 'MALE',
 'gender_unknown',
 'gender_other',
 '0-4',
 '100+',
 '15-19',
 '20-24',
 '25-29',
 '30-34',
 '35-39',
 '40-44',
 '45-49',
 '5-9',
 '50-54',
 '55-59',
 '60-64',
 '65-69',
 '70-74',
 '75-79',
 '80-84',
 '85-89',
 '90-94',
 '95-99',
 'age_NA',
 'basic',
 'facebook',
 'google',
 'ca',
 'cs',
 'da',
 'de',
 'el',
 'en',
 'es',
 'fi',
 'fr',
 'hr',
 'hu',
 'id',
 'is',
 'it',
 'ja',
 'ko',
 'nl',
 'no',
 'pl',
 'pt',
 'ru',
 'sv',
 'th',
 'tr',
 'zh',
 'api',
 'content',
 'direct',
 'other',
 'remarketing',
 'sem-brand',
 'sem-non-brand',
 'seo',
 'baidu',
 'bing',
 'craigslist',
 'daum',
 'email-marketing',
 'facebook-open-graph',
 'gsp',
 'meetup',
 'naver',
 'padmapper',
 'vast',
 'wayn',
 'yahoo',
 'yandex',
 'linked',
 'local ops',
 'marketing',
 'omg',
 'product',
 'tracked-other',
 'untracked',
 'Android',
 '

In [377]:
train_x.head()

,signup_flow,sum_secs_elapsed,counts,lat_destination,lng_destination,distance_km,destination_km2,language_levenshtein_distance,FEMALE,MALE,...,lag_account_created_first_active_before,lag_account_created_first_active_greater1,lag_account_created_first_active_sameday,deu,eng,fra,ita,nld,por,spa
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,NaN,NaN,36.966427,-95.84403,0.0,9826675.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [380]:
train_y = train_m.loc[:,'country_destination']

In [36]:
#EDA
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [63]:
sessions.groupby('action').agg({'secs_elapsed':'sum'}).sort_values('secs_elapsed', ascending=False)

,secs_elapsed
action,
show,3.653625e+10
index,2.465579e+10
update,1.390981e+10
search_results,1.300936e+10
confirm_email,1.171823e+10
search,8.465963e+09
active,7.898227e+09
campaigns,7.038574e+09
personalize,4.958936e+09


In [30]:
sum(pd.isnull(train.date_first_booking))

124543

In [25]:
test.iloc[0:5,:]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,lag_account_created_first_active,sum_secs_elapsed,counts
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaN,FEMALE,35-39,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,before,119187.0,8.0
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaN,-unknown-,NA,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,before,250119.0,19.0
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaN,-unknown-,NA,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,before,975575.0,58.0
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaN,-unknown-,NA,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,before,123001.0,11.0
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaN,-unknown-,NA,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,before,454023.0,19.0


In [45]:
print 'test shape ', test.shape
print 'train shape', train.shape

print '# test ids in sessions/#test ids', 61668.0/62096
print '# train ids in sessions/#train ids', 73815.0/213451

test shape  (62096, 15)
train shape (213451, 16)
# test ids in sessions/#test ids 0.993107446534
# train ids in sessions/#train ids 0.345817072771


In [ ]:
#delete all  but one time row now that we have lag times?
#remove either train['lag_account_created'] or train['lag_first_active'] to take into account leakage
#note country destination still in training

In [ ]:
#appendix showing the missinginess of the training ids in the sessions csv

In [26]:
strgroupids = ' '.join(group_sessions.user_id) #making a huge string of all the users ids in group_sesssions

In [36]:
sum(map(lambda x: strgroupids.find(x) != -1, test.id))

61668

In [37]:
sum(map(lambda x: strgroupids.find(x) != -1, train.id))

73815

In [75]:
!pip install xgboost

    100% |████████████████████████████████| 757kB 584kB/s 
  Running setup.py bdist_wheel for xgboost ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /Users/Zi/Library/Caches/pip/wheels/d3/43/37/f902e214730441ba23bfc73621fad90dd6634e7fb34090a804
Successfully built xgboost
You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [77]:
!pip install --upgrade pip

  Using cached pip-8.1.2-py2.py3-none-any.whl
  Found existing installation: pip 8.1.1
    Uninstalling pip-8.1.1:
      Successfully uninstalled pip-8.1.1


In [378]:
import sklearn.metrics
import time

def evaluate_model(clf):
    """Scores a model using log loss with the created train and test sets."""
    start = time.time()
    clf.fit(train_x, train_y)
    print "Train score:", sklearn.metrics.roc_auc_score(train_y, clf.predict_proba(train_x)[:,1])
    #print "Test score:", sklearn.metrics.roc_auc_score(test_y, clf.predict_proba(test_x)[:,1])
    print "Total time:", time.time() - start
    print

In [379]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=10)
evaluate_model(xgb)

Train score:

ValueError: multiclass format is not supported